<a href="https://colab.research.google.com/github/PmasCastro/Ufo_analysis/blob/main/Ufo_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Section




In [ ]:
!pip install folium openpyxl
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!pip install nbstripout
!nbstripout --install
!nbstripout --attributes 'metadata.widgets'

In [ ]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import ipywidgets as widgets
from IPython.display import display
import os

# --- Step 1: Load the data ---
file_name = "ufo_sightings_scrubbed.xlsx"

if not os.path.exists(file_name):
    print(f"❌ Error: File '{file_name}' not found in {os.getcwd()}")
    df = None
else:
    df = pd.read_excel(file_name)
    df.columns = df.columns.str.strip().str.lower()
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    df = df.dropna(subset=['latitude', 'longitude'])
    print(f"✅ Data loaded and cleaned. Total records: {len(df)}")

# --- Step 2: Setup filters & UI ---
if df is not None and not df.empty:
    country_options = ['All'] + sorted(df['country'].dropna().unique().tolist()) if 'country' in df.columns else ['All']
    shape_options = ['All'] + sorted(df['shape'].dropna().unique().tolist()) if 'shape' in df.columns else ['All']

    country_dropdown = widgets.Dropdown(options=country_options, description='Country:')
    shape_dropdown = widgets.Dropdown(options=shape_options, description='Shape:')
    ui = widgets.VBox([country_dropdown, shape_dropdown])

    # --- Step 3: Map function ---
    def show_map(country='All', shape='All'):
        filtered_df = df

        if country != 'All':
            filtered_df = filtered_df[filtered_df['country'] == country]
        if shape != 'All':
            filtered_df = filtered_df[filtered_df['shape'] == shape]

        if filtered_df.empty:
            print("No data matches the selected filters.")
            return

        # Limit the number of points for performance
        filtered_df = filtered_df.head(1000)

        # Create folium map centered roughly over US (adjust if needed)
        m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
        cluster = MarkerCluster().add_to(m)

        for _, row in filtered_df.iterrows():
            popup = (
                f"{row.get('datetime', 'N/A')}<br>"
                f"{row.get('city', 'N/A')}, {row.get('state', 'N/A')}<br>"
                f"Shape: {row.get('shape', 'N/A')}<br>"
                f"{row.get('comments', 'N/A')}"
            )
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=popup,
                icon=folium.Icon(color="purple", icon="info-sign")
            ).add_to(cluster)

        display(m)

    # --- Step 4: Display UI with interactive output ---
    interactive_out = widgets.interactive_output(show_map, {'country': country_dropdown, 'shape': shape_dropdown})
    display(ui, interactive_out)

else:
    print("❌ Map not created. Please check your data file and try again.")